In [1]:
import numpy as np
import random as rn
import findspark
import sys
import time
import numba
from numba import cuda, float32, float64
#findspark.init("/opt/spark")
findspark.init("C:\\Users\\Software\\spark-3.3.1-bin-hadoop3\\bin\\..")
from pyspark import SparkContext
sc = SparkContext("local","PSO Distribuido")

In [2]:
def MSE(y, pred):
    n = len(y)
    if n != len(pred):
        print("error: datos y predicción de distintos tamaños")
        return -1

    resultado = 0.0

    for i in range(n):
        resultado += (y[i] - pred[i])**2

    resultado = resultado / n

    return resultado

In [3]:
#PARÁMETROS del problema
V_max= 5.0
W= 1.0
c_1 = 0.8
c_2 = 0.2
objetivo =[50,50,50]
ind=0
filas_x=[0,0,0]
#Número dimensiones de los vectores
n=3
#Número de partículas
m= 10
#Número de iteraciones
I = 100

In [4]:
#Estructuas de datos
posiciones_=[]
mejor_posiciones_locales_=[]
velocidades_=[]
particulas = np.array([posiciones_,velocidades_,mejor_posiciones_locales_])
mejor_pos_global_=[]
mejor_pos_global_arr= np.array([0 for j in range(n)])
best_local_fitness_= []
best_local_fitness_arr=np.array(best_local_fitness_)
#maximum float
max = sys.float_info.max
best_global_fitness = max
accum= sc.accumulator(0)
mejor_pos_global= sc.broadcast(mejor_pos_global_arr)
best_local_fitness= sc.broadcast(best_local_fitness_arr)

In [5]:
def InitParticles(part,N,M,bgf,blf):
    #global best_global_fitness
    global accum
    parts_ =[]
    for j in range(M):
        posicion=[rn.uniform(-100,100) for j in range(N)]
        velocidad=[rn.uniform(-5,5) for j in range(N)]
        part_ = posicion + velocidad + posicion
        parts_.append(part_) 
        fit = MSE(posicion,objetivo)
        best_local_fitness_.append(fit)
        blf= np.array(best_local_fitness_)
        #print("bgf-> ",bgf)
        if fit < bgf:
            bgf = fit
            accum= fit
            mejor_pos_global = posicion
    parts = np.array(parts_)
    return blf,bgf,mejor_pos_global,parts

In [6]:
def fitnessEval(part,mpg,blf,bgf,N):
    global ind
    global accum
    filas=part[ind][0:N]
    fit = MSE(filas,objetivo)
    if fit < blf[ind]:
        blf[ind] = fit
        #mpl[ind] = part[0:1,ind]
        for k in range(N):
            part[2*N+k]=filas[k]
        if fit < bgf:
            bgf = fit
            mpg = filas
    accum= fit
    if ind<10:
        ind+=1
    #return part,mpg,blf,bgf
    return part

In [7]:
best_local_fitness,best_global_fitness,mejor_pos_global,particulas=InitParticles(particulas,n,m,best_global_fitness,best_local_fitness)
print("particulas:", particulas)
print("best local fitness:",best_local_fitness)
print("best global fitness:",best_global_fitness)
rdd= sc.parallelize(particulas)
rdd.foreach(lambda x:fitnessEval(particulas,mejor_pos_global,best_local_fitness,best_global_fitness,n))
print("best global fitness:",accum)

particulas: [[ 85.28886114 -83.25785796  60.7958735   -1.9076368   -1.36911552
   -4.13175826  85.28886114 -83.25785796  60.7958735 ]
 [-57.88830491 -85.98225339 -42.78158185  -1.17091112  -3.45818144
    2.59143661 -57.88830491 -85.98225339 -42.78158185]
 [-63.04262696  67.54275851  -8.19877577   3.43954932  -2.87586602
    2.01181374 -63.04262696  67.54275851  -8.19877577]
 [-68.33946396 -57.49952923 -43.72990859   1.44102444   0.82095558
    4.20352911 -68.33946396 -57.49952923 -43.72990859]
 [-40.23230329  39.86334926  68.09114728   0.49244197  -2.131253
   -1.77072022 -40.23230329  39.86334926  68.09114728]
 [ 51.44177202  12.36241928 -28.29288027  -4.56276305  -2.44761163
    3.33149894  51.44177202  12.36241928 -28.29288027]
 [ 99.0066676   51.97612784 -35.17358442   2.26029091  -3.9160282
    2.67508121  99.0066676   51.97612784 -35.17358442]
 [ 93.38910771  75.76680155  15.32489003  -3.6842113   -3.35867325
    1.64924542  93.38910771  75.76680155  15.32489003]
 [ 78.97191219 